In [1]:
import p_utils as pu
import pandas as pd
import json
from tqdm import tqdm 
import my_utils as mu
import numpy as np
import matplotlib.pylab as plt
import pandas as pd
import requests
from PIL import Image
from io import BytesIO 
import cv2
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression #로지스틱스 함수(모델생성)
from sklearn.preprocessing import StandardScaler #정규화
from sklearn.model_selection import train_test_split #학습세트/평가세트 분리
import traceback
import random

In [2]:
def getData():
    import p_utils as pu
    import pandas as pd
    import json
    from tqdm import tqdm 
    
    print("--1/6-- rawdata 불러오기 시작")
    query1 = 'select * from rawdata'
    df1 = pu.oracle_totalExecute(query1)
    
    print("--2/6-- summoner 불러오기 시작")
    query2 = 'select * from summoner'
    df2 = pu.oracle_totalExecute(query2)
    
    #아이디/티어 df 생성 
    df3 = df2.drop(columns=['SUMMONER_ID',
     'SUMMONER_PUUID',
     'API_KEY',
     'SUMMONER_LEVEL',
     'SUMMONER_PROFILE',
     'SUMMONER_WINS',
     'SUMMONER_LOSSES',
     'SUMMONER_VETERAN',
     'SUMMONER_INACTIVE',
     'SUMMONER_FRESHBLOOD',
     'SUMMONER_HOTSTREAK'])
    df3 = df3.rename(columns={'SUMMONER_NAME':'PARTICIPANT_NAME'})
    
    df_1 = []
    df_k = list(df1.keys())
    print("--3/6-- 데이터 변환 시작")
    for i in tqdm(range(len(df1))):
        lst = []
        for j in range(len(df_k)):
            if j < 10 :
                lst.append(df1[df_k[j]][i])
            else :
                data = df1[df_k[j]][i]
                data = data.replace("'", "\"")
                json_data = json.loads(data)
                lst.append(json_data)
        df_1.append(lst)
            
    df_t = pd.DataFrame(df_1, columns = df_k)
    
    df_2 = pd.merge(df_t,df3)
    

    print("--4/6-- train 데이터로 변환")
    df_3 = []
    for i in tqdm(range(len(df_2))):
        lst=[]
        
        lst.append(df_2['GAME_ID'][i])
        lst.append(df_2['PARTICIPANT_NAME'][i])
        lst.append(df_2['PARTICIPANT_NUMBER'][i])
        lst.append(df_2['CHAMPION'][i]['win'])
        
        wp = df_2['VISION'][i]['wardsPlaced']
        cwp = df_2['VISION'][i]['controlWardsPlaced']
        wk = df_2['VISION'][i]['wardsKilled']
        vision = wp+cwp+wk 
        lst.append(vision)
        
        cs = df_2['CS'][i]['totalMinionsKilled']
        time = round(df_2['GAME_DURATION'][i]/60)
        cpm = cs/time
        lst.append(cpm)
        
        lst.append(df_2['GAME_DURATION'][i])
        
        lst.append(df_2['KDA'][i]['kda'])
        
        lst.append(df_2['SUMMONER_TIER'][i])
        
        df_3.append(lst)
        
    df_4 = pd.DataFrame(df_3, columns = ['GAME_ID','PARTICIPANT_NAME','PARTICIPANT_NUMBER','WIN','VISION','CPM','GAME_DURATINO','KDA','SUMMONER_TIER'])
    
    # WIN int로 타입 변환
    df_4 = df_4.astype({'WIN':int})
    
    # kda null값 제거 
    df_4 = df_4.dropna(subset=['KDA'], how='any',axis=0)
    
    # unranked 티어 제거 
    remove = df_4[df_4['SUMMONER_TIER']=='Unranked'].index
    df_4 = df_4.drop(remove)
    
#     # 머신런닝-로지스틱스 회귀 
#     print("--5/6-- 머신런닝-로지스틱스 회귀 시작 ")

#     dft_k= list(df['SUMMONER_TIER'].unique())
#     dft_k
#     j=0
#     for i in dft_k:
#         if j==0:
#             df_input = df_4[df_4['SUMMONER_TIER']==i][0:250].reset_index().drop(columns=['index'])
#         else:
#             df_con = df_4[df_4['SUMMONER_TIER']==i][0:250].reset_index().drop(columns=['index'])
#             df_input = pd.concat([df_input,df_con])
#         j+=1

#     tier_input = df_4[['PARTICIPANT_NUMBER','WIN','VISION','CPM','GAME_DURATINO','KDA']].to_numpy()
#     tier_target = df_4['SUMMONER_TIER'].to_numpy()
#     train_input, test_input, train_target, test_target = train_test_split(tier_input, tier_target, random_state=42)
    
#     print("--5/6-- 머신런닝에 데이터 입력 ")
#     X_train, X_test, y_train, y_test = train_test_split(tier_input, tier_target, random_state=42)
#     dft_1=[]
#     df_4['SCORE'] = 1 
#     logreg = LogisticRegression()
#     logreg.fit(X_train, y_train)
#     for i in tqdm(range(5)):
#         test_arr=list(df_4[['PARTICIPANT_NUMBER','WIN','VISION','CPM','GAME_DURATINO','KDA']].iloc[0])
#         test_input = np.asarray([test_arr])
#         X_test = test_input.reshape(-1, len(test_arr))
#         y_pred = logreg.predict(X_test)
        
#         string = y_pred[0]
#         df_4['SCORE'][i] = string

#     df_4[:10]
            
    return df_4

In [3]:
df = getData()

--1/6-- rawdata 불러오기 시작
oracle open!
oracle close!
--2/6-- summoner 불러오기 시작
oracle open!
oracle close!
--3/6-- 데이터 변환 시작


100%|████████████████████████████████████████████████████████████████████████| 364900/364900 [02:38<00:00, 2298.30it/s]


--4/6-- train 데이터로 변환


100%|████████████████████████████████████████████████████████████████████████| 325201/325201 [01:08<00:00, 4770.69it/s]


In [4]:
df[:1]

,GAME_ID,PARTICIPANT_NAME,PARTICIPANT_NUMBER,WIN,VISION,CPM,GAME_DURATINO,KDA,SUMMONER_TIER
0,KR_6437691335,나영주니,7,0,15,1.21875,1944,3.8,PLATINUM


In [5]:
dft_k= list(df['SUMMONER_TIER'].unique())
j=0
for i in dft_k:
    if j==0:
        df_input = df[df['SUMMONER_TIER']==i][0:250].reset_index().drop(columns=['index'])
    else:
        df_con = df[df['SUMMONER_TIER']==i][0:250].reset_index().drop(columns=['index'])
        df_input = pd.concat([df_input,df_con])
    j+=1

In [6]:
tier_input = df_input[['PARTICIPANT_NUMBER','WIN','VISION','CPM','GAME_DURATINO']].to_numpy()

In [7]:
tier_target = df_input['SUMMONER_TIER'].to_numpy()

In [8]:
train_input, test_input, train_target, test_target = train_test_split(tier_input, tier_target, random_state=42)

In [9]:
ss = StandardScaler()
ss.fit(train_input)
train_scaled = ss.transform(train_input)
test_scaled = ss.transform(test_input)

In [10]:
kn = KNeighborsClassifier(n_neighbors=3)
kn.fit(train_scaled, train_target)

print(kn.score(train_scaled, train_target))
print(kn.score(test_scaled, test_target))

0.4889896373056995
0.1883495145631068


In [53]:
X_train, X_test, y_train, y_test = train_test_split(tier_input, tier_target, random_state=42)

In [102]:
import random

In [110]:
test_df['PARTICIPANT_NUMBER'].isnull().sum()

0

In [113]:
test_df.iloc[35]

PARTICIPANT_NUMBER    10.000000
WIN                    1.000000
VC                    53.085714
KDA                    9.500000
Name: 36, dtype: float64

In [119]:
import traceback
import random

In [144]:
test_df.keys()

Index(['PARTICIPANT_NUMBER', 'WIN', 'VC', 'KDA'], dtype='object')

In [149]:
df2['SCORE'] = 'GOLD'

In [11]:
logreg = LogisticRegression()
X_train = tier_input
y_train = tier_target
logreg.fit(X_train, y_train)

df['SCORE'] = None
for i in tqdm(range(len(df))):
    test_arr=list(df[['PARTICIPANT_NUMBER','WIN','VISION','CPM','GAME_DURATINO']].iloc[i])
    test_input = np.asarray([test_arr])
    X_test = test_input.reshape(-1, len(test_arr))
    y_pred = logreg.predict(X_test)
    
    string = y_pred[0]
    df.at[i, 'SCORE'] = string
    
df_test[:10]

100%|█████████████████████████████████████████████████████████████████████████| 316130/316130 [51:22<00:00, 102.57it/s]


NameError: name 'df_test' is not defined

In [12]:
df[:10]

,GAME_ID,PARTICIPANT_NAME,PARTICIPANT_NUMBER,WIN,VISION,CPM,GAME_DURATINO,KDA,SUMMONER_TIER,SCORE
0,KR_6437691335,나영주니,7.0,0.0,15.0,1.218750,1944.0,3.800000,PLATINUM,IRON
1,KR_6437691335,유콩순,8.0,0.0,7.0,6.781250,1944.0,1.500000,PLATINUM,IRON
2,KR_6376915251,유콩순,8.0,0.0,11.0,7.068966,1730.0,2.200000,PLATINUM,GRANDMASTER
3,KR_6437691335,pastell,9.0,0.0,13.0,7.093750,1944.0,1.833333,PLATINUM,PLATINUM
4,KR_6479378528,pastell,9.0,0.0,6.0,6.655172,1760.0,2.000000,PLATINUM,IRON
5,KR_6437691335,막죽어도되는계정,10.0,0.0,40.0,2.625000,1944.0,2.200000,PLATINUM,MASTER
6,KR_6488080797,막죽어도되는계정,5.0,0.0,24.0,3.291667,1466.0,2.000000,PLATINUM,GOLD
7,KR_6437691335,똥싸는마린,3.0,1.0,18.0,6.843750,1944.0,1.666667,PLATINUM,PLATINUM
8,KR_6437691335,strip show,4.0,1.0,11.0,7.093750,1944.0,2.000000,PLATINUM,PLATINUM
9,KR_6437691335,쿨감폿,5.0,1.0,23.0,1.218750,1944.0,3.400000,PLATINUM,BRONZE


In [13]:
df.to_csv('mlResult.csv')

In [138]:
df_test = df_test.drop(columns=['SCORE'])

In [148]:
df_test['SCORE'].unique()

array(['SILVER'], dtype=object)

In [28]:
df['SCORE'].iloc[0][0]

'PLATINUM'

In [128]:
test_df.iloc[295670]

PARTICIPANT_NUMBER     2.000000
WIN                    1.000000
VC                    32.868421
KDA                   10.800000
Name: 325188, dtype: float64

In [146]:
df2[['PARTICIPANT_NUMBER','WIN','VC','KDA']].iloc[0]

PARTICIPANT_NUMBER     7.00000
WIN                    0.00000
VC                    16.21875
KDA                    7.80000
Name: 0, dtype: float64

In [99]:
test_arr=list(df2[['PARTICIPANT_NUMBER','WIN','VC','KDA']].iloc[0])
test_input = np.asarray([test_arr])
X_test = test_input.reshape(-1, len(test_arr))

logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)

In [100]:
y_pred

array(['GOLD'], dtype=object)

In [149]:
df_test = df.drop(columns = ['SCORE'])
df_test[:1]

,GAME_ID,PARTICIPANT_NAME,PARTICIPANT_NUMBER,WIN,VISION,CPM,GAME_DURATINO,KDA,SUMMONER_TIER
0,KR_6437691335,나영주니,7,0,15,1.21875,32,3.8,PLATINUM


In [66]:
dft_k= list(df_test['SUMMONER_TIER'].unique())
dft_k

['PLATINUM',
 'GOLD',
 'DIAMOND',
 'SILVER',
 'MASTER',
 'BRONZE',
 'IRON',
 'GRANDMASTER',
 'CHALLENGER']

In [71]:
a = df_test[df_test['SUMMONER_TIER']=='PLATINUM'][0:500].reset_index().drop(columns=['index']) 

In [101]:
b = df_test[df_test['SUMMONER_TIER']=='GOLD'][0:500].reset_index().drop(columns=['index']) 

In [102]:
result = pd.concat([a,b])
result

,GAME_ID,PARTICIPANT_NAME,PARTICIPANT_NUMBER,WIN,VISION,CPM,GAME_DURATINO,KDA,SUMMONER_TIER
0,KR_6437691335,나영주니,7,0,15,1.218750,32,3.800000,PLATINUM
1,KR_6437691335,유콩순,8,0,7,6.781250,32,1.500000,PLATINUM
2,KR_6376915251,유콩순,8,0,11,7.068966,29,2.200000,PLATINUM
3,KR_6437691335,pastell,9,0,13,7.093750,32,1.833333,PLATINUM
4,KR_6479378528,pastell,9,0,6,6.655172,29,2.000000,PLATINUM
...,...,...,...,...,...,...,...,...,...
495,KR_6462999384,1561651651561,3,0,9,6.700000,30,1.571429,GOLD
496,KR_6419594608,지민이는 공주야,3,0,6,5.750000,16,0.200000,GOLD
497,KR_6463319461,죽기시작,4,1,7,6.750000,16,8.000000,GOLD
498,KR_6476350238,죽기시작,10,0,39,0.222222,27,1.555556,GOLD


In [75]:
len(dft_k) *500

4500

In [114]:
dft_k= list(df_test['SUMMONER_TIER'].unique())
dft_k
j=0
for i in dft_k:
    if j==0:
        df_input = df_test[df_test['SUMMONER_TIER']==i][0:250].reset_index().drop(columns=['index'])
    else:
        df_con = df_test[df_test['SUMMONER_TIER']==i][0:250].reset_index().drop(columns=['index'])
        df_input = pd.concat([df_input,df_con])
    j+=1

In [115]:
df_input

,GAME_ID,PARTICIPANT_NAME,PARTICIPANT_NUMBER,WIN,VISION,CPM,GAME_DURATINO,KDA,SUMMONER_TIER
0,KR_6437691335,나영주니,7,0,15,1.218750,32,3.800000,PLATINUM
1,KR_6437691335,유콩순,8,0,7,6.781250,32,1.500000,PLATINUM
2,KR_6376915251,유콩순,8,0,11,7.068966,29,2.200000,PLATINUM
3,KR_6437691335,pastell,9,0,13,7.093750,32,1.833333,PLATINUM
4,KR_6479378528,pastell,9,0,6,6.655172,29,2.000000,PLATINUM
...,...,...,...,...,...,...,...,...,...
54,KR_6420041586,카시는작고소중해,1,1,19,7.375000,24,5.000000,CHALLENGER
55,KR_6450088131,쿨탱이,9,1,8,8.062500,16,18.000000,CHALLENGER
56,KR_6408132887,At1anta,4,1,16,7.818182,22,12.000000,CHALLENGER
57,KR_6408814074,헤어지기 싫어,3,1,24,5.000000,29,8.000000,CHALLENGER


In [84]:
len(df_input)

59